--- Day 11: Space Police ---

On the way to Jupiter, you're pulled over by the Space Police.

"Attention, unmarked spacecraft! You are in violation of Space Law! All spacecraft must have a clearly visible registration identifier! You have 24 hours to comply or be sent to Space Jail!"

Not wanting to be sent to Space Jail, you radio back to the Elves on Earth for help. Although it takes almost three hours for their reply signal to reach you, they send instructions for how to power up the emergency hull painting robot and even provide a small Intcode program (your puzzle input) that will cause it to paint your ship appropriately.

There's just one problem: you don't have an emergency hull painting robot.

You'll need to build a new emergency hull painting robot. The robot needs to be able to move around on the grid of square panels on the side of your ship, detect the color of its current panel, and paint its current panel black or white. (All of the panels are currently black.)

The Intcode program will serve as the brain of the robot. The program uses input instructions to access the robot's camera: provide 0 if the robot is over a black panel or 1 if the robot is over a white panel. Then, the program will output two values:

    First, it will output a value indicating the color to paint the panel the robot is over: 0 means to paint the panel black, and 1 means to paint the panel white.
    Second, it will output a value indicating the direction the robot should turn: 0 means it should turn left 90 degrees, and 1 means it should turn right 90 degrees.

After the robot turns, it should always move forward exactly one panel. The robot starts facing up.

The robot will continue running for a while like this and halt when it is finished drawing. Do not restart the Intcode computer inside the robot during this process.

For example, suppose the robot is about to start running. Drawing black panels as ., white panels as #, and the robot pointing the direction it is facing (< ^ > v), the initial state and region near the robot looks like this:
```
.....
.....
..^..
.....
.....
```
The panel under the robot (not visible here because a ^ is shown instead) is also black, and so any input instructions at this point should be provided 0. Suppose the robot eventually outputs 1 (paint white) and then 0 (turn left). After taking these actions and moving forward one panel, the region now looks like this:
```
.....
.....
.<#..
.....
.....
```
Input instructions should still be provided 0. Next, the robot might output 0 (paint black) and then 0 (turn left):
```
.....
.....
..#..
.v...
.....
```
After more outputs (1,0, 1,0):
```
.....
.....
..^..
.##..
.....
```
The robot is now back where it started, but because it is now on a white panel, input instructions should be provided 1. After several more outputs (0,1, 1,0, 1,0), the area looks like this:
```
.....
..<#.
...#.
.##..
.....
```
Before you deploy the robot, you should probably have an estimate of the area it will cover: specifically, you need to know the number of panels it paints at least once, regardless of color. In the example above, the robot painted 6 panels at least once. (It painted its starting panel twice, but that panel is still only counted once; it also never painted the panel it ended on.)

Build a new emergency hull painting robot and run the Intcode program on it. How many panels does it paint at least once?


In [1]:
from copy import deepcopy
from collections import deque

class IntcodeComputer(object):
    
    def __init__(self, program, mem_size = 2048):
        self.mem = deepcopy(program)
        self.mem += ['0'] * (mem_size - len(self.mem))
        self.i = 0
        self.rb = 0
        self.inputs = deque([])
        self.last_out = None
        self.done = False
        
    def process_instruction(self, instruction):
        instruction = instruction.zfill(2)
        opcode = instruction[-2:]
        param_modes = instruction[:-2]
        
        if opcode == '01':
            self._add(param_modes)
        elif opcode == '02':
            self._mult(param_modes)
        elif opcode == '03':
            self._input(param_modes)
        elif opcode == '04':
            return self._output(param_modes)
        elif opcode == '05':
            self._jump_if_true(param_modes)
        elif opcode == '06':
            self._jump_if_false(param_modes)
        elif opcode == '07':
            self._less_than(param_modes)
        elif opcode == '08':
            self._equals(param_modes)
        elif opcode == '09':
            self._update_rb(param_modes)
        elif opcode == '99':
            self.done = True
            return 0
        else:
            raise ValueError(f"Unknown opcode: '{opcode}' from instruction: '{instruction}'")
        
    def _get_param_val(self, param_pos, mode):
        if mode == '0':
            return int(self.mem[int(self.mem[self.i+param_pos])])
        elif mode == '1':
            return int(self.mem[self.i+param_pos])
        elif mode == '2':
            return int(self.mem[int(self.mem[self.i+param_pos]) + self.rb])
        else:
            raise ValueError(f'Unknown mode: {mode}')
        
    def _write_param(self, param_pos, mode, value):
        if mode == '0':
            self.mem[int(self.mem[self.i+param_pos])] = str(value)
        elif mode == '1':
            self.mem[self.i+param_pos] = str(value)
        elif mode == '2':
            self.mem[int(self.mem[self.i+param_pos]) + self.rb] = str(value)
        else:
            raise ValueError(f'Unknown mode: {mode}')

        
    def _add(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        self._write_param(3, param_modes[-3], a+b)
        self.i += 4
    
    def _mult(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        self._write_param(3, param_modes[-3], a*b)
        self.i += 4
    
    def _input(self, param_modes):
        param_modes = param_modes.zfill(1)
        self._write_param(1, param_modes[-1], self.inputs.popleft())
        self.i += 2
            
    def _output(self, param_modes):
        param_modes = param_modes.zfill(1)
        out = self._get_param_val(1, param_modes[-1])
        self.i += 2
        self.last_out = out
        return out
        
    def _jump_if_true(self, param_modes):
        param_modes = param_modes.zfill(2)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a != 0:
            self.i = b
        else:
            self.i += 3
            
    def _jump_if_false(self, param_modes):
        param_modes = param_modes.zfill(2)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a == 0:
            self.i = b
        else:
            self.i += 3  
            
    def _less_than(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a < b:
            self._write_param(3, param_modes[-3], 1)
        else:
            self._write_param(3, param_modes[-3], 0)
        self.i += 4
        
    def _equals(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a == b:
            self._write_param(3, param_modes[-3], 1)
        else:
            self._write_param(3, param_modes[-3], 0)
        self.i += 4
        
    def _update_rb(self, param_modes):
        param_modes = param_modes.zfill(1)
        a = self._get_param_val(1, param_modes[-1])
        self.rb += a
        self.i += 2
        
    def run(self):
        while self.i < len(self.mem):
            out = self.process_instruction(self.mem[self.i])
            if out is not None:
                return out


class HullPaintingRobot(object):
    
    FACING_TABLE = [(0, 1),  # up
                    (1, 0),  # right
                    (0, -1), # down
                    (-1, 0)] # left
    
    def __init__(self, code, start=(0,0), start_color = 0):
        self.cpu = IntcodeComputer(code)
        self.facing = 0
        self.position = start
        self.path = [start]
        self.tiles = {(start): start_color}
        
    def _move_bot(self):
        px, py = self.position
        mx, my = self.FACING_TABLE[self.facing]
        self.position = (px+mx, py+my)
        self.path.append(self.position)
        
    def _turn_bot(self, direction):
        if direction == 0:
            if self.facing == 0:
                self.facing = 3
            else:
                self.facing -= 1
        elif direction == 1:
            # turn right
            if self.facing == 3:
                self.facing = 0
            else:
                self.facing += 1
        else:
            raise ValueError(f'Invalid turn direction: {direction}')
        
    def _paint_panel(self, color):
        self.tiles[self.position] = color
                
    def _read_tile(self):
        if not self.position in self.tiles:
            self.tiles[self.position] = 0
        return self.tiles[self.position]
            
    def _move(self):
        self.cpu.inputs.append(self._read_tile())
        self._paint_panel(self.cpu.run())
        self._turn_bot(self.cpu.run())
        self._move_bot()
        
    def run(self):
        while not self.cpu.done:
            self._move()
            
        return len(self.tiles)
    
    def render(self):
        max_x = max(list(self.tiles.keys()), key=lambda i: i[0])[0]
        max_y = max(list(self.tiles.keys()), key=lambda i: i[1])[1]
        min_x = min(list(self.tiles.keys()), key=lambda i: i[0])[0]
        min_y = min(list(self.tiles.keys()), key=lambda i: i[1])[1]
        
        for y in reversed(range(min_y, max_y+1)):
            for x in range(min_x, max_x+1):
                if (x, y) not in self.tiles:
                    print_char = ' '
                else:
                    if self.tiles[(x, y)] == 0:
                        print_char = ' '
                    else:
                        print_char = '#'
                print(print_char, end='')
            print('')       

In [2]:
# Part 1
program = open('input/day11.txt').read().splitlines()[0].split(',')

hpr = HullPaintingRobot(program, start_color = 0)
hpr.run()

1932

--- Part Two ---

You're not sure what it's trying to paint, but it's definitely not a registration identifier. The Space Police are getting impatient.

Checking your external ship cameras again, you notice a white panel marked "emergency hull painting robot starting panel". The rest of the panels are still black, but it looks like the robot was expecting to start on a white panel, not a black one.

Based on the Space Law Space Brochure that the Space Police attached to one of your windows, a valid registration identifier is always eight capital letters. After starting the robot on a single white panel instead, what registration identifier does it paint on your hull?


In [3]:
hpr2 = HullPaintingRobot(program, start_color = 1)
hpr2.run()
hpr2.render()

 ####  ##  #  # #  #  ##    ## #### ###    
 #    #  # #  # # #  #  #    # #    #  #   
 ###  #    #### ##   #       # ###  #  #   
 #    # ## #  # # #  # ##    # #    ###    
 #    #  # #  # # #  #  # #  # #    # #    
 ####  ### #  # #  #  ###  ##  #### #  #   
